Notebook which processes the h5 data file produced for a rocking curve scan at fixed time delay at the beamline XPP at the Linear Coherent Light Source. The notebook outputs the rocking curve for laser on and laser off

In [ ]:
### load modules
import h5py
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy import signal

In [ ]:
### load the h5 file
f = h5py.File('/data3/2021/2021_3_LCLS_LAO/smalldata/xpplv7918_Run0029.h5','r')

In [ ]:
#### extract data fields
ipm2 = np.array(f['ipm2/sum'])
detec = np.array(f['jungfrau1M/ROIw_0_area'])
WP = np.array(f['epics/las_comp_wp'])
xOn = np.array(f['lightStatus/xray'])
laserOn = np.array(f['lightStatus/laser'])
lasdelay = np.array(f['enc/lasDelay'])
omega = np.array(f['scan/xpp_gon_kappa_e_chi'])

In [ ]:
print(np.nanstd(lasdelay)) ## standard deviation not including NANs. if this is less than 1 the time tool was not used

6.029741790692631e-05


In [ ]:
### define a detector threshold and sort shots into laser on and laser off
thresh = 20.0
#ind = np.array(np.where(xOn== int(0)) and np.where(laserOn == int(0)) and np.where(ipm2 > thresh))
ind_off = np.where(np.logical_and(np.logical_and(xOn==1, laserOn==0), ipm2>thresh))
ind_on = np.where(np.logical_and(np.logical_and(xOn==1, laserOn==1), ipm2>thresh))

### normalization
ipm_on = ipm2[ind_on]
ipm_off = ipm2[ind_off]

### detector images 
detec_off = detec_roi[ind_off]
detec_on = detec_roi[ind_on]

### omegas of the rocking curve
omegas_off = omega[ind_off]
omegas_on = omega[ind_on]


In [ ]:
### arrays for detector mean for on and off omegas of the rocking scans
mean_on = np.zeros(len(np.unique(omega)))
mean_off = np.zeros(len(np.unique(omega)))

### region of interest for each unique omega, for laser on and off
totalArray_on = np.zeros((len(np.unique(omega)), detec_on.shape[1], detec_on.shape[2]))#len(tau_Delays))
totalArray_off = np.zeros((len(np.unique(omega)), detec_off.shape[1], detec_off.shape[2]))

### condending the omegas of every shot to unique omegas for laser on and off
uniq_omegas_on = np.unique(omega)
uniq_omegas_off = np.unique(omega)

### calculating the average detector value for laser on
for index in range(len(mean_on)):
    avg_ipm_on = np.mean(ipm_on[np.where(omegas_on == uniq_omegas_on[index])])
    mean_on[index]=np.mean(detec_on[np.where(omegas_on == uniq_omegas_on[index])])/avg_ipm_on
    totalArray_on[index] = np.mean(detec_on[np.where(omegas_on == uniq_omegas_on[index])], axis=0)/avg_ipm_on

### calculating the average detector value for laser off
for index1 in range(len(mean_off)):
    avg_ipm_off = np.mean(ipm_off[np.where(omegas_off == uniq_omegas_off[index1])])
    mean_off[index1]=np.mean(detec_off[np.where(omegas_off == uniq_omegas_off[index1])])/avg_ipm_off
    totalArray_off[index1]=np.mean(detec_off[np.where(omegas_off == uniq_omegas_off[index1])], axis=0)/avg_ipm_off



In [ ]:
### plotting the laser on and laser off rocking curves
%matplotlib notebook
plt.plot(uniq_omegas_on, mean_on, '-o', color='black')
plt.plot(uniq_omegas_off, mean_off, '-o', color='red')

In [ ]:
### save the laser on rocking curve
with open('29Array_On', 'wb') as f:
    np.save(f, totalArray_on)

In [ ]:
### save the laser off rocking curve
with open('29Array_Off', 'wb') as f:
    np.save(f, totalArray_off)